# 🚀 Evalscope Toolkit - 模型评估

[![GitHub](https://img.shields.io/badge/GitHub-evalscope--toolkit-blue?logo=github)](https://github.com/maxuan1798/evalscope-toolkit)
[![License](https://img.shields.io/badge/License-MIT-green.svg)](https://github.com/maxuan1798/evalscope-toolkit/blob/main/LICENSE)

这是一个简化的模型评估 notebook，基于 [Evalscope Toolkit](https://github.com/maxuan1798/evalscope-toolkit) 开源项目。

## ✨ 特性

- 📦 **开箱即用**：自动下载依赖和工具包
- 🎯 **简单配置**：只需设置模型和数据集即可
- 🔄 **自动化**：自动管理 vLLM 服务和评估流程
- 📊 **多数据集**：支持 10+ 种评估数据集

## 🎯 支持的数据集

`gsm8k` (数学) | `humaneval` (代码) | `mmlu` (通识) | `competition_math` | `drop` | `hellaswag` | `arc` | `truthfulqa` | `winogrande` | `math_500`

## 📋 快速开始

1. **运行下面的单元格** - 自动安装依赖
2. **配置模型和数据集** - 修改配置单元格
3. **运行评估** - 一键开始

---

## 1. 安装依赖和下载工具包

⚠️ **首次运行时请执行此单元格**

此单元格将：
- 从 GitHub 克隆整个仓库（包含 `evalscope_toolkit` 工具包）
- 自动安装所需的依赖（torch, vllm, evalscope 等）

> 💡 也可以手动克隆仓库：
> ```bash
> git clone https://github.com/maxuan1798/evalscope_toolkit.git
> cd evalscope_toolkit
> ```

In [ ]:
import sys
import subprocess
from pathlib import Path

# GitHub 仓库配置
GITHUB_REPO = "https://github.com/maxuan1798/evalscope_toolkit.git"

def verify_toolkit(toolkit_path):
    """验证工具包是否完整"""
    required_files = [
        '__init__.py',
        'config.py',
        'dataset_manager.py',
        'vllm_service.py',
        'evaluator.py',
        'utils.py',
    ]
    
    for filename in required_files:
        if not (toolkit_path / filename).exists():
            return False
    return True

def setup_environment():
    """设置环境：克隆仓库和安装依赖"""
    print("="*60)
    print("环境设置")
    print("="*60)
    
    # 检查工具包是否存在
    toolkit_path = Path.cwd() / "evalscope_toolkit"
    
    # 如果已存在且完整，跳过下载
    if toolkit_path.exists() and verify_toolkit(toolkit_path):
        print("✓ evalscope_toolkit 已存在且完整")
    else:
        # 下载仓库
        print("\n📦 克隆 evalscope_toolkit 仓库...")
        print(f"仓库地址: {GITHUB_REPO}")
        
        try:
            # 清理旧文件（如果存在但不完整）
            if toolkit_path.exists():
                print("清理旧的不完整文件...")
                import shutil
                shutil.rmtree(toolkit_path)
            
            # 克隆仓库到父目录的临时位置
            result = subprocess.run(
                ['git', 'clone', '--depth', '1', GITHUB_REPO, 'evalscope_toolkit_repo'],
                check=True,
                capture_output=True,
                text=True
            )
            
            # 移动 evalscope_toolkit 文件夹到当前目录
            import shutil
            repo_path = Path.cwd() / "evalscope_toolkit_repo"
            src_toolkit = repo_path / "evalscope_toolkit"
            
            if src_toolkit.exists():
                shutil.move(str(src_toolkit), str(toolkit_path))
                print("✓ 工具包下载成功")
            else:
                print("⚠ 仓库中未找到 evalscope_toolkit 文件夹")
                return False
            
            # 清理临时仓库
            if repo_path.exists():
                shutil.rmtree(repo_path)
                
        except subprocess.CalledProcessError as e:
            print(f"⚠ Git 克隆失败: {e}")
            print("\n请手动克隆仓库：")
            print(f"  git clone {GITHUB_REPO}")
            print(f"  然后将 evalscope_toolkit 文件夹复制到当前目录")
            return False
        except Exception as e:
            print(f"⚠ 下载失败: {e}")
            print("\n请手动克隆仓库：")
            print(f"  git clone {GITHUB_REPO}")
            return False
    
    # 添加到 Python 路径
    cwd_str = str(Path.cwd())
    if cwd_str not in sys.path:
        sys.path.insert(0, cwd_str)
    
    # 验证可以导入
    try:
        import evalscope_toolkit
        print("✓ 成功导入 evalscope_toolkit")
    except ImportError as e:
        print(f"⚠ 无法导入 evalscope_toolkit: {e}")
        return False
    
    # 安装依赖
    print("\n📦 安装依赖...")
    try:
        from evalscope_toolkit.utils import setup_dependencies
        if setup_dependencies():
            print("\n✓ 环境设置完成！")
            return True
        else:
            print("\n⚠ 部分依赖安装失败，请检查错误信息")
            return False
    except Exception as e:
        print(f"\n⚠ 安装依赖失败: {e}")
        import traceback
        traceback.print_exc()
        return False

# 执行环境设置
if setup_environment():
    print("\n" + "="*60)
    print("✓ 准备就绪！请继续下一步配置")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠ 环境设置遇到问题，请检查上面的错误信息")
    print("="*60)

## 2. 配置模型和数据集

在这里配置您要评估的模型和数据集。

In [ ]:
import sys
from pathlib import Path

# 确保当前目录在 Python 路径中
cwd = str(Path.cwd())
if cwd not in sys.path:
    sys.path.insert(0, cwd)

# 重新加载模块（如果之前已加载）
if 'evalscope_toolkit' in sys.modules:
    import importlib
    importlib.reload(sys.modules['evalscope_toolkit'])

from evalscope_toolkit import EvalConfig

# ============================================================
# 配置区域 - 请根据您的需求修改以下配置
# ============================================================

# 模型配置（可以是多个模型，用列表形式）
MODELS = [
    "unsloth/Llama-3.2-3B-Instruct",  # HuggingFace 模型 ID
    # "/path/to/local/model",          # 或本地模型路径
]

# 数据集配置（可以是多个数据集）
DATASETS = [
    "gsm8k",           # 数学推理
    # "humaneval",     # 代码生成
    # "mmlu",          # 多任务语言理解
]

# GPU 配置
GPUS = "0"              # 使用的 GPU ID，例如 "0" 或 "0,1"
GPU_MEM_UTIL = 0.6      # GPU 内存利用率 (0.0-1.0)

# 评估参数（可选，使用默认值即可）
EVAL_BATCH_SIZE = 32    # 评估批次大小
MAX_NEW_TOKENS = 2048   # 最大生成 token 数
TEMPERATURE = 0.0       # 采样温度

# ============================================================
# 创建配置对象
# ============================================================

config = EvalConfig(
    models=MODELS,
    datasets=DATASETS,
    gpus=GPUS,
    gpu_memory_utilization=GPU_MEM_UTIL,
    eval_batch_size=EVAL_BATCH_SIZE,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
)

print("配置信息：")
print("="*60)
print(f"模型: {config.models}")
print(f"数据集: {config.datasets}")
print(f"GPU: {config.gpus}")
print(f"GPU 内存利用率: {config.gpu_memory_utilization}")
print(f"数据目录: {config.data_root}")
print(f"日志目录: {config.log_root}")
print("="*60)
print("✓ 配置完成！")

## 3. 查看支持的数据集

运行此单元格可以查看所有支持的数据集及其描述。

In [ ]:
import sys
from pathlib import Path

# 确保路径正确
if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

from evalscope_toolkit import DatasetManager

print("支持的数据集：")
print("="*60)

for dataset_name in DatasetManager.list_supported_datasets():
    info = DatasetManager.get_dataset_info(dataset_name)
    print(f"\n📊 {dataset_name}")
    print(f"   {info['description']}")
    print(f"   ModelScope: {info['ms_name']}")
    if info['subset_name']:
        print(f"   Subset: {info['subset_name']}")
    print(f"   Split: {info['split']}")

print("\n" + "="*60)

## 4. 运行评估

执行此单元格开始评估。评估过程包括：
1. 下载和准备数据集
2. 启动 vLLM 服务
3. 运行评估
4. 生成结果报告

⚠️ **注意**：评估可能需要较长时间，具体取决于模型大小和数据集数量。

In [ ]:
import sys
from pathlib import Path

# 确保路径正确
if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

from evalscope_toolkit import Evaluator

# 创建评估器
evaluator = Evaluator(config)

# 运行评估
try:
    results = evaluator.run()
    print("\n" + "="*60)
    print("✓ 评估完成！")
    print("="*60)
except Exception as e:
    print(f"\n⚠ 评估失败: {e}")
    import traceback
    traceback.print_exc()

## 5. 查看评估结果

查看评估结果摘要和详细信息。

In [ ]:
# 显示评估结果
evaluator.show_results()

## 6. 详细结果分析（可选）

查看每个模型和数据集的详细评估结果。

In [ ]:
import json
from pathlib import Path

# 读取评估摘要
summary_file = evaluator.base_log_dir / "evaluation_summary.json"

if summary_file.exists():
    with open(summary_file) as f:
        summary = json.load(f)
    
    print("评估摘要：")
    print("="*60)
    print(f"实例 ID: {summary['instance_id']}")
    print(f"用户 ID: {summary['user_id']}")
    print(f"日志目录: {summary['log_dir']}")
    print("\n模型评估结果：")
    
    for model_name, model_results in summary['models'].items():
        print(f"\n📊 模型: {model_name}")
        
        if 'error' in model_results:
            print(f"   ⚠ 错误: {model_results['error']}")
            continue
        
        print(f"   总耗时: {model_results.get('total_duration', 0)} 秒")
        print("   数据集结果：")
        
        for dataset_name, dataset_results in model_results.get('datasets', {}).items():
            print(f"\n   - {dataset_name}:")
            print(f"     耗时: {dataset_results['duration']} 秒")
            print(f"     开始时间: {dataset_results['start_time']}")
            print(f"     结束时间: {dataset_results['end_time']}")
            print(f"     结果目录: {dataset_results['work_dir']}")
    
    print("\n" + "="*60)
else:
    print("未找到评估摘要文件")

## 📖 使用说明

### 如何添加自己的模型：
```python
MODELS = [
    "unsloth/Llama-3.2-3B-Instruct",  # HuggingFace 模型
    "/path/to/your/local/model",      # 本地模型
]
```

### 如何选择数据集：
```python
DATASETS = [
    "gsm8k",        # 数学推理
    "humaneval",    # 代码生成
    "mmlu",         # 通用知识
]
```

### GPU 配置：
- 单 GPU: `GPUS = "0"`
- 多 GPU: `GPUS = "0,1"`
- CPU 模式: `GPUS = ""`

### 结果位置：
所有评估结果保存在 `log/outputs_<用户>_<实例ID>/` 目录下。

### 常见问题：
1. **端口被占用**：程序会自动寻找可用端口
2. **GPU 内存不足**：降低 `GPU_MEM_UTIL` 值
3. **模型下载慢**：使用本地模型或设置 HuggingFace 镜像

### 获取帮助：
如遇到问题，请查看日志文件或提交 GitHub Issue。